In [1]:
# Common modules
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, auc, roc_curve
from sklearn.feature_selection import mutual_info_classif

# Decision tree specific modules
from sklearn import tree

from scipy.sparse import load_npz

from sklearn.feature_extraction.text import CountVectorizer #Can use tfidffvectorizer as well
import pandas as pd 
import os
import re
import numpy as np

from nltk.stem import PorterStemmer
from matplotlib import pyplot as plt
from collections import defaultdict, Counter

In [2]:
BASE_PATH = os.path.join(os.getcwd(), os.pardir)
DATA_PATH = os.path.join(BASE_PATH, 'data')

In [3]:
X_train = load_npz(os.path.join(DATA_PATH, 'training_feats.npz'))

In [4]:
X_test = load_npz(os.path.join(DATA_PATH, 'test_feats.npz'))

In [5]:
X_train.shape

(49308, 35520)

In [6]:
X_test.shape

(74659, 35520)

In [7]:
train_df = pd.read_json(os.path.join(BASE_PATH, '01-milestone1', 'imputed_train.json'))
test_df =  pd.read_json(os.path.join(DATA_PATH, 'test.json.zip'))

In [8]:
y_train = train_df['interest_level']

In [9]:
X_test_indexes = test_df.index

# Preprocessing
As described in milestone 1, we will do some feature extraction.


In [10]:
# def preprocess(train_df, min_feats=5):
#     bathrooms = train_df['bathrooms']
#     bedrooms = train_df['bedrooms']
#     building_ids = train_df['building_id']
#     latitudes = train_df['latitude']
#     longitudes = train_df['longitude']
#     manager_ids = train_df['manager_id']
#     prices = train_df['price']
    
#     datetime = pd.to_datetime(train_df['created'])
    
#     months = datetime.dt.month
#     days = datetime.dt.day
#     hours = datetime.dt.hour
    
#     # Where Monday = 0, and Sunday = 6
#     weekdays = datetime.dt.dayofweek
    
#     num_photos = train_df['photos'].str.len()
    
#     features = train_df['features'].apply(lambda x: [i.lower() for i in x])

#     feature_counts = Counter()
#     for feature in features.tolist():
#         feature_counts.update(feature)
#     feature = sorted([k for (k,v) in feature_counts.items() if v > min_feats])
    
#     key2original = defaultdict(list)
#     k = 4
#     for f in feature:
#         cleaned = clean(f)
#         key = cleaned[:k].strip()
#         key2original[key].append(f)
    
#     columns = list(key2original.keys())
    
#     # reverse hash table
#     original2key = {}
#     for col in columns:
#         for original in key2original[col]:
#             original2key[original] = col
            
#     all_listing_features = {}

#     for index,row in train_df.iterrows():
#         listing_features = {}
#         features_found = []
#         for feature in row['features']:
#             feature = feature.lower()
#             if feature in original2key:
#                 features_found.append(original2key[feature])
#         for feature in columns:
#                 if feature not in features_found:
#                     listing_features[feature] = 0
#                 else:
#                     listing_features[feature] = 1
#         all_listing_features[row['listing_id']] = listing_features

#     one_hot_features = pd.DataFrame.from_dict(all_listing_features, orient='index')
    
#     # Description attribute
    
#     descriptions = train_df[['description']]
#     # Removes symbols, numbers and stem the words to reduce dimentional space
#     stemmer = PorterStemmer()
    
#     descriptions['description_new'] = descriptions.description.apply(lambda x: clean_description(x, stemmer))

#     cvect_desc = CountVectorizer(stop_words='english', max_features=200)
#     full_sparse = cvect_desc.fit_transform(descriptions.description_new)

#     # Renaming words to avoid collisions with other feature names in the model
#     col_desc = ['desc_'+ i for i in cvect_desc.get_feature_names()] 
#     count_vect_df = pd.DataFrame(full_sparse.todense(), columns=col_desc)
#     descriptions = pd.concat([descriptions.reset_index(), count_vect_df],axis=1)
    
#     descriptions = descriptions.drop(labels=['description', 'index', 'description_new'], axis=1)
#     descriptions.index = train_df['listing_id']
    
#     # The final dataframe to be returned
#     final_train_df = pd.DataFrame()
    
#     final_train_df['bathrooms'] = bathrooms
#     final_train_df['bedrooms'] = bedrooms
#     final_train_df['building_ids'] = building_ids 
#     final_train_df['latitudes'] = latitudes
#     final_train_df['longitudes'] = longitudes
#     final_train_df['manager_ids'] = manager_ids
#     final_train_df['prices'] = prices
#     final_train_df['months'] = months
#     final_train_df['days'] = days
#     final_train_df['hours'] = hours
#     final_train_df['weekdays'] = weekdays
#     final_train_df['num_photos'] = num_photos
#     final_train_df.index = train_df['listing_id']
    
#     final_train_df = final_train_df.merge(descriptions, left_index=True, right_index=True)
#     final_train_df = final_train_df.merge(one_hot_features, left_index=True, right_index=True)
    
#     final_train_df = pd.concat([final_train_df, pd.get_dummies(final_train_df['building_ids'], prefix='building')], axis=1)
#     final_train_df = pd.concat([final_train_df, pd.get_dummies(final_train_df['manager_ids'], prefix='manager')], axis=1)
#     final_train_df = final_train_df.drop(['building_ids', 'manager_ids'], axis=1)

#     return final_train_df
    

In [11]:
# def clean(s):
#     x = s.replace("-", "")
#     x = x.replace(" ", "")
#     x = x.replace("twenty four hour", "24")
#     x = x.replace("24/7", "24")
#     x = x.replace("24hr", "24")
#     x = x.replace("24-hour", "24")
#     x = x.replace("24hour", "24")
#     x = x.replace("24 hour", "24")
#     x = x.replace("common", "cm")
#     x = x.replace("concierge", "doorman")
#     x = x.replace("bicycle", "bike")
#     x = x.replace("private", "pv")
#     x = x.replace("deco", "dc")
#     x = x.replace("decorative", "dc")
#     x = x.replace("onsite", "os")
#     x = x.replace("outdoor", "od")
#     x = x.replace("ss appliances", "stainless")
#     return x

# def clean_description(x, stemmer):
#     regex = re.compile('[^a-zA-Z ]')
#     # For user clarity, broken it into three steps
#     i = regex.sub(' ', x).lower()
#     i = i.split(" ") 
#     i= [stemmer.stem(l) for l in i]
#     i= " ".join([l.strip() for l in i if (len(l)>2) ]) # Keeping words that have length greater than 2
#     return i

# def feature_hash(x):
#     cleaned = clean(x, uniq)
#     key = cleaned[:4].strip()
#     return key

## Training the model

We first instantiate a decision tree model and train it naively using all the features.

In [12]:
# Convert labels from {low, medium high} -> {0, 1, 2}
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

## Train-test split

We need to split the training data into a training set and a validation set (to evaluate the performance of the model).

For now we do 5-fold cross_validation.

In [13]:
seed = 36201431

Our initial approach is to use all features for training. For the parameters, we simply use the defaults.

In [15]:
model = tree.DecisionTreeClassifier()
scores = cross_validate(model, X_train, y_train, scoring='neg_log_loss', cv=5)
# Convert negative log loss to log loss
test_scores = -1 * scores['test_score']
print("Log loss score:")
print(test_scores.mean().round(4))

Log loss score:
11.1711


Now we can try training on the entire train set. We will obtain a model, train it on the test dataset, then submit to kaggle for evaluation.

In [16]:
model = tree.DecisionTreeClassifier(criterion='gini')
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [17]:
y_pred = model.predict_proba(X_test)

In [18]:
le.classes_

array(['high', 'low', 'medium'], dtype=object)

Since the order is not correct (low corresponds to the label 1, but in the csv they expect the low to be the third column), we must swap columns for y_pred.

There may be a "bug" below, I don't think I actually swapped the columns yet.

In [19]:
def create_submission_csv(y_pred, X_test_indexes):
    df = pd.DataFrame(y_pred, columns=le.classes_)
    df.index = X_test_indexes
    df.index.name = 'listing_id'
    return df

In [20]:
output = create_submission_csv(y_pred, X_test_indexes)
output.to_csv('decision_tree_predictions_1.csv')

# Score from Kaggle
9.18820 <- ignore, this is the old score

## Attempt 2

It seems that using all the features is not very good. Let's use a smaller set of features.
We can choose the top x features in order of descending mutual information (wrt the label).

Some additional things to try:

- entropy for information gain
- limiting max_depth to like 30 or so
- class_weight = balanced
- ccp_alpha = ??? 

In [22]:
n_feats = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 35520]

In [23]:
cross_val_scores = {}

In [27]:
mutual_info = mutual_info_classif(X_train, y_train, random_state=seed)

In [32]:
n_feat = 50
top_features = mutual_info.argsort()[-1 * n_feat:][::-1]

# Only use these top n features when training, to eliminate overfitting.
X_train_final = X_train[:,top_features]

model = tree.DecisionTreeClassifier()
scores = cross_validate(model, X_train_final, y_train, scoring='neg_log_loss', cv=5, return_train_score=True)
# Convert negative log loss to log loss
test_scores = -1 * scores['test_score']
train_scores = -1 * scores['train_score']
score = test_scores.mean().round(4)
train_score = train_scores.mean().round(4)
print("Mean log loss test  score using top {0} features: {1}".format(n_feat, score))
print("Mean log loss train score using top {0} features: {1}".format(n_feat, train_score))

Mean log loss test  score using top 50 features: 11.9486
Mean log loss train score using top 50 features: 0.0013


In [31]:
best_score = 9999999999999999
best_n = 0

for n_feat in n_feats:
    print("Considering top {0} features.".format(n_feat))
    top_features = mutual_info.argsort()[-1 * n_feat:][::-1]

    # Only use these top n features when training, to eliminate overfitting.
    X_train_final = X_train[:,top_features]
    
    model = tree.DecisionTreeClassifier()
    scores = cross_validate(model, X_train_final, y_train, scoring='neg_log_loss', cv=5)
    # Convert negative log loss to log loss
    test_scores = -1 * scores['test_score']
    score = test_scores.mean().round(4)
    print("Mean log loss score using top {0} features: {1}".format(n_feat, score))
    
    
    cross_val_scores[n_feat] = score
    
    if score < best_score:
        best_score = score
        best_n = n_feat

print("The best number of features is {0}".format(best_n))
print("The log loss for this number of features is {0}".format(best_score))

Considering top 10 features.
Mean log loss score using top 10 features: 12.1066
Considering top 20 features.
Mean log loss score using top 20 features: 11.4989
Considering top 50 features.
Mean log loss score using top 50 features: 11.9752
Considering top 100 features.
Mean log loss score using top 100 features: 11.7936
Considering top 200 features.
Mean log loss score using top 200 features: 11.7696
Considering top 500 features.
Mean log loss score using top 500 features: 11.65
Considering top 1000 features.
Mean log loss score using top 1000 features: 11.6352
Considering top 2000 features.
Mean log loss score using top 2000 features: 11.4958
Considering top 5000 features.
Mean log loss score using top 5000 features: 11.3704
Considering top 10000 features.
Mean log loss score using top 10000 features: 11.2833
Considering top 35520 features.
Mean log loss score using top 35520 features: 11.1823
The best number of features is 35520
The log loss for this number of features is 11.1823


In [ ]:
# Train on entire dataset
X_train_final = X_test[:,best_n]
X_test_final = X_test[:,best_n]

model = tree.DecisionTreeClassifier(criterion='gini')
model.fit(X_train_final, y_train)

y_pred = model.predict_proba(X_test_final)

output = create_submission_csv( y_pred, X_test_indexes)
output.to_csv('decision_tree_predictions_2.csv')